## Path Settings

In [1]:
import os
OPENPOSE_PATH="./openpose/"
HOME_PATH='./'

## Installing OpenPose

In [2]:
#@title
!pip install ffmpeg-python
from os.path import exists, join, basename, splitext

# see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
# install new CMake becaue of CUDA10
!wget -q https://cmake.org/files/v3.17/cmake-3.17.2-Linux-x86_64.tar.gz
!tar xfz cmake-3.17.2-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

# install system dependencies
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
!rm -rf openpose
# clone openpose
!git clone -q --depth 1 $git_repo_url
# --recursive necessary in the line below, as otherwise you can (sometimes) get "lpthreads" errors in cmake ("undefined reference to `pthread_create'" etc). See, for example, https://github.com/facebookarchive/caffe2/issues/1234
!sed -i 's/execute_process(COMMAND git checkout --recursive master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
!cd openpose && git submodule update --init --recursive --remote

!wget "https://drive.usercontent.google.com/download?id=1QCSxJZpnWvM00hx49CJ2zky7PWGzpcEh&export=download&authuser=0&confirm=t&uuid=50355e54-0445-4875-9d0e-3866a1f6239d&at=APZUnTVA3sHsK5rk-u2O6phIms2L%3A1711987634035" -O models.zip

# !cp "/content/drive/MyDrive/VTO/openpose/models.zip" "/content/"
!unzip models.zip -d openpose

Submodule '3rdparty/caffe' (https://github.com/CMU-Perceptual-Computing-Lab/caffe.git) registered for path '3rdparty/caffe'
Submodule '3rdparty/pybind11' (https://github.com/pybind/pybind11.git) registered for path '3rdparty/pybind11'
Cloning into '/content/openpose/3rdparty/caffe'...
Cloning into '/content/openpose/3rdparty/pybind11'...
Submodule path '3rdparty/caffe': checked out '2d4bf549aabf97265f76b9599b82540dde9218ed'
Submodule path '3rdparty/pybind11': checked out '19a6b9f4efb569129c878b7f8db09132248fbaa1'
--2024-05-05 07:12:36--  https://drive.usercontent.google.com/download?id=1QCSxJZpnWvM00hx49CJ2zky7PWGzpcEh&export=download&authuser=0&confirm=t&uuid=50355e54-0445-4875-9d0e-3866a1f6239d&at=APZUnTVA3sHsK5rk-u2O6phIms2L%3A1711987634035
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.172.132, 2607:f8b0:4001:c61::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.172.132|:443... connected.
HTTP request sent, await

In [3]:
! sed -i 's/executeShInItsFolder "getModels.sh"/# executeShInItsFolder "getModels.sh"/g' ./openpose/scripts/ubuntu/install_openpose_JetsonTX2_JetPack3.1.sh
! sed -i 's/executeShInItsFolder "getModels.sh"/# executeShInItsFolder "getModels.sh"/g' ./openpose/scripts/ubuntu/install_openpose_JetsonTX2_JetPack3.3.sh
! sed -i 's/download_model("BODY_25"/# download_model("BODY_25"/g' ./openpose/CMakeLists.txt
! sed -i 's/78287B57CF85FA89C03F1393D368E5B7/# 78287B57CF85FA89C03F1393D368E5B7/g' ./openpose/CMakeLists.txt
! sed -i 's/download_model("body (COCO)"/# download_model("body (COCO)"/g' ./openpose/CMakeLists.txt
! sed -i 's/5156d31f670511fce9b4e28b403f2939/# 5156d31f670511fce9b4e28b403f2939/g' ./openpose/CMakeLists.txt
! sed -i 's/download_model("body (MPI)"/# download_model("body (MPI)"/g' ./openpose/CMakeLists.txt
! sed -i 's/2ca0990c7562bd7ae03f3f54afa96e00/# 2ca0990c7562bd7ae03f3f54afa96e00/g' ./openpose/CMakeLists.txt
! sed -i 's/download_model("face"/# download_model("face"/g' ./openpose/CMakeLists.txt
! sed -i 's/e747180d728fa4e4418c465828384333/# e747180d728fa4e4418c465828384333/g' ./openpose/CMakeLists.txt
! sed -i 's/download_model("hand"/# download_model("hand"/g' ./openpose/CMakeLists.txt
! sed -i 's/a82cfc3fea7c62f159e11bd3674c1531/# a82cfc3fea7c62f159e11bd3674c1531/g' ./openpose/CMakeLists.txt

## Make (takes \~ 10 minutes!)

In [4]:
!cd openpose && rm -rf build || true && mkdir build

In [5]:
cmake_file='/content/openpose/CMakeLists.txt'
!cd openpose && sed -i 's/-DBUILD_python=OFF/-DBUILD_python=ON/g' $cmake_file
!cd openpose && sed -i 's/-DBUILD_python_layer=OFF/-DBUILD_python_layer=ON/g' $cmake_file

!cd openpose && sed -i 's/option(BUILD_PYTHON "Build OpenPose python." OFF)/option(BUILD_PYTHON "OpenPose python." ON)\noption(BUILD_BIN_FOLDER "Copy 3rd-party DLL files." ON)/g' $cmake_file

In [6]:
!cd openpose && cd build && cmake .. -DUSE_CUDNN=OFF -DGENERATE_PYTHON_BINDINGS:BOOL="1" -DPYTHON_LIBRARY='/usr/lib/x86_64-linux-gnu/libpython3.10.so' && make -j`nproc`

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc - works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ - works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- GCC detected, adding compile flags
-- GCC detected, adding compile flags
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE  
-- Found CUDA: /usr/local/cuda (found version "12.2") 
-- Building with CUDA.
-- CUDA detected: 12.2
-- Added CUDA NVCC flags for: sm_75
-- Found cuDNN

## Mount drive with dataset

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Make the build executable

In [8]:
!cd $OPENPOSE_PATH && chmod -R 755 './build/'

## Extract frames for all videos, pass them through openpose, save the keypoints locally and delete everything else

In [12]:
os.mkdir("/content/output_jsons/")

In [ ]:
import os
import subprocess
from google.colab import files
path = "/content/drive/MyDrive/ataxia_dataset"
for video in os.listdir(path):
  print(video)
  if ".mp4" not in video:
    continue
  else:
    # Break the video into frames at 30 fps
    command = ["ffmpeg", "-i", os.path.join(path, video), "-vf", "fps=30", "/content/frames/output_%05d.jpg"]
    out = subprocess.run(command, stderr=subprocess.PIPE)
    # Extract keypoints using openpose (COCO / OG)
    openposed = ["./build/examples/openpose/openpose.bin", "--image_dir", "/content/frames/", "--write_json", "/content/output_jsons/", "--display", "0", "--render_pose", "0", "--model_pose", "BODY_25"]
    openpoze = subprocess.run(openposed, stderr=subprocess.PIPE, cwd="/content/openpose")
    # Zip keypoints for this video
    zipper = ["zip", f"/content/keypoints_{video[:-4]}.zip", "/content/output_jsons/"]
    subprocess.run(zipper)
    # Download the keypoints
    files.download(f"/content/keypoints_{video[:-4]}.zip")

Anonymized_ratings.csv
ratings_file_with_source_to_anonymized_mapping.csv
60.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

99.mp4


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

20.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

129.mp4


## Thank you!